In [7]:
import MySQLdb

In [12]:
db = MySQLdb.connect(host="localhost",  # host      # 127.0.0.1
                     user="root",       # username  # root
                     passwd="1234",     # password  # 1234
                     db="mydb")         # DB NAME   

In [13]:
# 데이터베이스 커서(Cursor)는 일련의 데이터에 순차적으로 액세스할 때 검색 및 "현재 위치"를 포함하는 데이터 요소이다.
cur = db.cursor()
# https://movie.naver.com/movie/bi/mi/basic.nhn?code=161967

In [39]:
# IF(조건, 참값, 거짓값)
query = """
        CREATE TABLE IF NOT EXISTS naver_movie
            (code    INTEGER      NOT NULL      PRIMARY KEY,
             url     VARCHAR(255) NOT NULL,
             created_at    DATETIME DEFAULT CURRENT_TIMESTAMP,
             title   VARCHAR(100) NOT NULL,
             story   TEXT,
             opened  DATETIME)
             ENGINE INNODB;
        """
cur.execute(query)


/Users/Mac/anaconda3/envs/lecture_env/lib/python3.7/site-packages/ipykernel_launcher.py:12: Warning: (1050, "Table 'naver_movie' already exists")
  if sys.path[0] == '':


0

In [40]:
query = """
        CREATE TABLE IF NOT EXISTS movie_comment(
            star    INTEGER    NOT NULL,
            comment TEXT       NOT NULL,
            movie_code INTEGER,
            author  VARCHAR(100),
            commented_at DATETIME,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            PRIMARY KEY (author, commented_at),
            FOREIGN KEY(movie_code)
            REFERENCES naver_movie(code) on delete cascade)
            ENGINE INNODB;
            ;
"""
cur.execute(query)

/Users/Mac/anaconda3/envs/lecture_env/lib/python3.7/site-packages/ipykernel_launcher.py:15: Warning: (1050, "Table 'movie_comment' already exists")
  from ipykernel import kernelapp as app


0

In [ ]:
# https://movie.naver.com/movie/bi/mi/basic.nhn?code=161967

In [ ]:
# https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=pnt&date=20190616

In [83]:
import requests
from urllib import parse
from bs4 import BeautifulSoup
import datetime
import math

In [184]:
# movie_list

# 한 화면에 50페이지
PAGE_SIZE = 50
MOVIE_LENGTH = 100
pages = math.ceil(MOVIE_LENGTH/PAGE_SIZE)
page

1

In [185]:
def get_movie_list(start_page, end_page):
    today = datetime.datetime.now()
    
    movie_list = []
    for page in range(start_page, end_page+1):
        movie_list_url = 'https://movie.naver.com/movie/sdb/rank/rmovie.nhn'
        params = {
            'sel':'pnt',
            'date':today.strftime('%Y%m%d'),
            'page':page
        }
        resp = requests.get(movie_list_url)
        resp.encoding = 'utf-8'
    
        soup = BeautifulSoup(resp.content)
        # list_ranking = soup.find(class_='list_ranking').find_all('tr')
        list_ranking = soup.select('.list_ranking>tbody>tr')
    
        for movie_tr in list_ranking:
            all_td = movie_tr.find_all('td')
            try:
                img = all_td[0].img['alt']
                link = all_td[1].find('a')['href']
                title = all_td[1].find('a').text
                
                code = parse.parse_qs(parse.urlsplit(link).query)['code'][0]
            except TypeError:
                pass

            else:
                movie_list.append({
                    'img':img,
                    'link':link,
                    'title':title,
                    'code':code
                })
                
    return movie_list
    
    

In [190]:
def get_movie_detail(code):
    base_url = 'https://movie.naver.com/movie/bi/mi/basic.nhn'
    params = {
        'code':code
    }
    resp = requests.get(base_url, params)
    resp.encoding ='utf-8'
    soup = BeautifulSoup(resp.content)
    
    opend = soup.find('dt', text='개요()').find_next_sibling('dd').find_all('span')[-1].text.strip()
    opend = ''.join(opend.split('\n'))[:10]

    story = soup.find('div', class_='story_area')
    print(story)
    return {
        'opend':opend,
        'story':story
    }
    
    
def get_movie_comment(code, last_comment='15788558'):
    base_url = '/movie/bi/mi/pointWriteFormList.nhn'
    page = 1
    comment_list = []
    
    while True:
        params={
            'code':171539,
            'type':'after',
            'order':'newest',
            'page':page
        }
        resp = requests.get(base_url, params)
        resp.encoding = 'utf-8'
        soup = BeautifulSoup(resp.content)
        
        comments = soup.find('div', class_='score_result').find_all('li')
        for comment in comments:
            star = comment.find('div', class_='star_score').text
            reple = comment.find('div', class_='score_reple').find('p').text
            
            dd = comment.find('div', class_='score_reple').find('dl').find_all('dd')
            user = dd[0]
            created_at = dd[1]
            
            comment_id = comment.find('div', class_='btn_area').find_all('span')[-1]['class'][0].split('_')[1]
            
            if int(comment_id) <= int(last_comment):
                return comment_list
            
            comment_list.append({
                'star':star,
                'reple':reple,
                'user': user,
                'created_at': created_at,
                'comment_id': comment_id
            })
            
        
        page += 1
        
    
    
    
    

In [191]:
movie_list = get_movie_list(1, pages)

In [192]:
movie_list

[{'img': '01',
  'link': '/movie/bi/mi/basic.nhn?code=161967',
  'title': '기생충',
  'code': '161967'},
 {'img': '02',
  'link': '/movie/bi/mi/basic.nhn?code=163788',
  'title': '알라딘',
  'code': '163788'},
 {'img': '03',
  'link': '/movie/bi/mi/basic.nhn?code=97631',
  'title': '맨 인 블랙: 인터내셔널',
  'code': '97631'},
 {'img': '04',
  'link': '/movie/bi/mi/basic.nhn?code=164125',
  'title': '엑스맨: 다크 피닉스',
  'code': '164125'},
 {'img': '05',
  'link': '/movie/bi/mi/basic.nhn?code=177371',
  'title': '0.0MHz',
  'code': '177371'},
 {'img': '06',
  'link': '/movie/bi/mi/basic.nhn?code=181704',
  'title': '로켓맨',
  'code': '181704'},
 {'img': '07',
  'link': '/movie/bi/mi/basic.nhn?code=18781',
  'title': '이웃집 토토로',
  'code': '18781'},
 {'img': '08',
  'link': '/movie/bi/mi/basic.nhn?code=136900',
  'title': '어벤져스: 엔드게임',
  'code': '136900'},
 {'img': '09',
  'link': '/movie/bi/mi/basic.nhn?code=183836',
  'title': '천로역정: 천국을 찾아서',
  'code': '183836'},
 {'img': '010',
  'link': '/movie/bi/mi/basi

In [193]:
get_movie_detail(171539)

<div class="story_area">
<div class="title_area">
<h4 class="h_story"><strong class="blind">줄거리</strong></h4>
</div>
<h5 class="h_tx_story">언제 어디서든 바른 생활! 완벽한 천재 뮤지션 ‘돈 셜리’<br/>
원칙보다 반칙! 다혈질 운전사 ‘토니’<br/>
취향도, 성격도 완벽히 다른 두 남자의 특별한 우정이 시작된다!</h5>
<br/> 그들을 위한 여행안내서 ‘그린북’에 의존해 특별한 남부 투어를 시작하는데…</p>
<button class="story_more" id="toggleMakingnoteButton" onclick="storyAndNote.toggleMakingnote();" type="button"><em class="blind">제작노트 보기</em></button><!-- N=a:mai.story -->
</div>


{'opend': '2019.01.09', 'story': <div class="story_area">
 <div class="title_area">
 <h4 class="h_story"><strong class="blind">줄거리</strong></h4>
 </div>
 <h5 class="h_tx_story">언제 어디서든 바른 생활! 완벽한 천재 뮤지션 ‘돈 셜리’<br/>
 원칙보다 반칙! 다혈질 운전사 ‘토니’<br/>
 취향도, 성격도 완벽히 다른 두 남자의 특별한 우정이 시작된다!</h5>
 <p class="con_tx">1962년 미국, 입담과 주먹만 믿고 살아가던 토니 발레롱가(비고 모텐슨)는 교양과 우아함 그 자체인
 <br/> 천재 피아니스트 돈 셜리(마허샬라 알리) 박사의 운전기사 면접을 보게 된다.
 <br/> 
 <br/> 백악관에도 초청되는 등 미국 전역에서 콘서트 요청을 받으며 명성을 떨치고 있는 돈 셜리는
 <br/> 위험하기로 소문난 미국 남부 투어 공연을 떠나기로 결심하고,
 <br/> 투어 기간 동안 자신의 보디가드 겸 운전기사로 토니를 고용한다.
 <br/> 
 <br/> 거친 인생을 살아온 토니 발레롱가와 교양과 기품을 지키며 살아온 돈 셜리 박사.
 <br/> 생각, 행동, 말투, 취향까지 달라도 너무 다른 두 사람은
 <br/> 그들을 위한 여행안내서 ‘그린북’에 의존해 특별한 남부 투어를 시작하는데…</p>
 <button class="story_more" id="toggleMakingnoteButton" onclick="storyAndNote.toggleMakingnote();" type="button"><em class="blind">제작노트 보기</em></button><!-- N=a:mai.story -->
 </div>}

In [194]:
resp = requests.get(movie_list_url)
resp.encoding = 'utf-8'
resp

<Response [200]>

In [195]:
# INSERT INTO table (column_list)
# VALUES (value_list)
# ON DUPLICATE KEY UPDATE
#    c1 = v1, 
#    c2 = v2,
#    ...;